In [6]:
import pandas as pd
from sklearn import  datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import  train_test_split
import mlflow
from mlflow.models import infer_signature


In [7]:
# set tracking uri
mlflow.set_tracking_uri(uri = "http://127.0.0.1:5000")

In [10]:
# load dataset
X,y = datasets.load_iris(return_X_y=True)
# split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# Define the model hyperparameters
params = {"penalty":"l2","solver": "lbfgs", "max_iter": 1000, "multi_class": "auto", "random_state": 8888}

##train the model

lr=LogisticRegression(**params)
lr.fit(X_train,y_train)

c:\Users\yazid\.conda\envs\mlflow\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888)

In [14]:
# prediction test set
y_pred = lr.predict(X_test)
y_pred

array([0, 1, 0, 1, 2, 0, 0, 2, 1, 1, 0, 0, 1, 1, 0, 2, 2, 2, 1, 1, 1, 0,
       1, 1, 1, 2, 1, 2, 2, 2])

In [34]:
accuracy = accuracy_score(y_test,y_pred)
print(f"{accuracy:.2f}")

0.90


In [27]:
## MLflow tracking
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
# create mlflow experimen
mlflow.create_experiment("lr_experiment")
# start mlflow run
with mlflow.start_run():
    # log parameters
    mlflow.log_params(params)
    # log accuracy metric
    mlflow.log_metric("accuracy", accuracy)
    # set tag as reminder
    mlflow.set_tag("model", "LR model, iris")
    # set infer signature is essential for defining and validating the input and output
    signature = infer_signature(X_train, lr.predict(X_train))
    # log the model
    model_info = mlflow.sklearn.log_model(
        sk_model = lr,
        artifact_path = "iris_model",
        signature = signature,
        input_example = X_train,
        registered_model_name = "tracking quick start"
    )

Successfully registered model 'tracking quick start'.
2024/12/17 20:02:38 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking quick start, version 1


🏃 View run fun-elk-787 at: http://127.0.0.1:5000/#/experiments/0/runs/4493d3dae80f4421907923f66332ef7c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


Created version '1' of model 'tracking quick start'.


## changing model parameters to compare it

In [42]:
# Define the model hyperparameters
params = {
    "penalty": "l1",
    "solver": "saga",
    "max_iter": 500,
    "multi_class": "ovr",
    "random_state": 42,
    "C": 0.5,
    "class_weight": "balanced",
    "fit_intercept": True,
    "tol": 1e-3
}

##train the model

lr=LogisticRegression(**params)
lr.fit(X_train,y_train)


c:\Users\yazid\.conda\envs\mlflow\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(C=0.5, class_weight='balanced', max_iter=500,
                   multi_class='ovr', penalty='l1', random_state=42,
                   solver='saga', tol=0.001)

In [43]:
y_pred=lr.predict(X_test)
y_pred

array([0, 1, 0, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 0, 2, 2, 1, 1, 1, 1, 0,
       1, 1, 1, 2, 1, 2, 2, 2])

In [44]:
accuracy=accuracy_score(y_test,y_pred)
print(f"{accuracy:.2f}")

0.93


In [45]:
## Sstart an MLFLOW run
with mlflow.start_run():
    ## log the hyperparameters
    mlflow.log_params(params)

    ## Log the accuracy metrics
    mlflow.log_metric("accuracy",accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    ## Infer the model signature

    signature=infer_signature(X_train,lr.predict(X_train))

    ## log the model
    model_info=mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart", # we use registred only when we chose best model

    )

Successfully registered model 'tracking-quickstart'.
2024/12/17 20:23:36 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 1


🏃 View run inquisitive-fowl-440 at: http://127.0.0.1:5000/#/experiments/0/runs/30986c85c72a44cbaea472c1977a2aba
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


Created version '1' of model 'tracking-quickstart'.


## Inferencing and validating the model

In [48]:
from mlflow.models import validate_serving_input

model_uri = model_info.model_uri

# The model is logged with an input example. MLflow converts
# it into the serving payload format for the deployed model endpoint,
# and saves it to 'serving_input_payload.json'
serving_payload = """{
  "inputs": [
    [
      4.4,
      3.2,
      1.3,
      0.2
    ],
    [
      5.2,
      3.5,
      1.5,
      0.2
    ],
    [
      6.5,
      3.0,
      5.5,
      1.8
    ],
    [
      5.0,
      3.5,
      1.3,
      0.3
    ],
    [
      6.7,
      3.1,
      5.6,
      2.4
    ],
    [
      5.7,
      2.6,
      3.5,
      1.0
    ],
    [
      6.2,
      2.8,
      4.8,
      1.8
    ],
    [
      5.7,
      4.4,
      1.5,
      0.4
    ],
    [
      7.7,
      3.0,
      6.1,
      2.3
    ],
    [
      5.9,
      3.2,
      4.8,
      1.8
    ],
    [
      4.8,
      3.1,
      1.6,
      0.2
    ],
    [
      6.1,
      2.8,
      4.0,
      1.3
    ],
    [
      4.8,
      3.4,
      1.9,
      0.2
    ],
    [
      5.8,
      2.6,
      4.0,
      1.2
    ],
    [
      5.7,
      2.8,
      4.5,
      1.3
    ],
    [
      4.4,
      2.9,
      1.4,
      0.2
    ],
    [
      5.1,
      3.5,
      1.4,
      0.3
    ],
    [
      6.1,
      3.0,
      4.9,
      1.8
    ],
    [
      5.4,
      3.7,
      1.5,
      0.2
    ],
    [
      6.7,
      3.1,
      4.4,
      1.4
    ],
    [
      4.9,
      3.0,
      1.4,
      0.2
    ],
    [
      4.6,
      3.2,
      1.4,
      0.2
    ],
    [
      4.3,
      3.0,
      1.1,
      0.1
    ],
    [
      7.9,
      3.8,
      6.4,
      2.0
    ],
    [
      4.6,
      3.1,
      1.5,
      0.2
    ],
    [
      5.0,
      2.0,
      3.5,
      1.0
    ],
    [
      6.0,
      3.4,
      4.5,
      1.6
    ],
    [
      5.4,
      3.9,
      1.3,
      0.4
    ],
    [
      7.7,
      2.8,
      6.7,
      2.0
    ],
    [
      5.4,
      3.9,
      1.7,
      0.4
    ],
    [
      5.6,
      2.8,
      4.9,
      2.0
    ],
    [
      7.1,
      3.0,
      5.9,
      2.1
    ],
    [
      6.4,
      2.7,
      5.3,
      1.9
    ],
    [
      5.7,
      2.5,
      5.0,
      2.0
    ],
    [
      5.1,
      3.7,
      1.5,
      0.4
    ],
    [
      5.9,
      3.0,
      5.1,
      1.8
    ],
    [
      6.7,
      3.0,
      5.2,
      2.3
    ],
    [
      6.8,
      3.0,
      5.5,
      2.1
    ],
    [
      6.0,
      2.2,
      5.0,
      1.5
    ],
    [
      5.0,
      3.2,
      1.2,
      0.2
    ],
    [
      6.3,
      3.4,
      5.6,
      2.4
    ],
    [
      5.8,
      2.7,
      5.1,
      1.9
    ],
    [
      6.5,
      3.2,
      5.1,
      2.0
    ],
    [
      5.1,
      3.3,
      1.7,
      0.5
    ],
    [
      5.5,
      3.5,
      1.3,
      0.2
    ],
    [
      5.8,
      2.8,
      5.1,
      2.4
    ],
    [
      5.8,
      4.0,
      1.2,
      0.2
    ],
    [
      5.0,
      3.4,
      1.5,
      0.2
    ],
    [
      6.4,
      2.9,
      4.3,
      1.3
    ],
    [
      5.4,
      3.0,
      4.5,
      1.5
    ],
    [
      4.6,
      3.6,
      1.0,
      0.2
    ],
    [
      6.8,
      3.2,
      5.9,
      2.3
    ],
    [
      6.3,
      2.5,
      5.0,
      1.9
    ],
    [
      5.0,
      3.5,
      1.6,
      0.6
    ],
    [
      5.1,
      3.8,
      1.6,
      0.2
    ],
    [
      6.5,
      3.0,
      5.8,
      2.2
    ],
    [
      7.7,
      2.6,
      6.9,
      2.3
    ],
    [
      5.1,
      2.5,
      3.0,
      1.1
    ],
    [
      6.4,
      2.8,
      5.6,
      2.1
    ],
    [
      6.7,
      3.3,
      5.7,
      2.1
    ],
    [
      6.2,
      2.9,
      4.3,
      1.3
    ],
    [
      4.9,
      2.4,
      3.3,
      1.0
    ],
    [
      6.2,
      3.4,
      5.4,
      2.3
    ],
    [
      4.7,
      3.2,
      1.6,
      0.2
    ],
    [
      5.5,
      4.2,
      1.4,
      0.2
    ],
    [
      7.2,
      3.6,
      6.1,
      2.5
    ],
    [
      6.3,
      3.3,
      6.0,
      2.5
    ],
    [
      5.6,
      3.0,
      4.5,
      1.5
    ],
    [
      5.5,
      2.5,
      4.0,
      1.3
    ],
    [
      6.0,
      3.0,
      4.8,
      1.8
    ],
    [
      4.9,
      3.1,
      1.5,
      0.2
    ],
    [
      4.4,
      3.0,
      1.3,
      0.2
    ],
    [
      6.3,
      2.7,
      4.9,
      1.8
    ],
    [
      5.0,
      3.4,
      1.6,
      0.4
    ],
    [
      6.9,
      3.2,
      5.7,
      2.3
    ],
    [
      5.6,
      2.7,
      4.2,
      1.3
    ],
    [
      7.0,
      3.2,
      4.7,
      1.4
    ],
    [
      6.7,
      2.5,
      5.8,
      1.8
    ],
    [
      4.8,
      3.0,
      1.4,
      0.1
    ],
    [
      5.7,
      3.8,
      1.7,
      0.3
    ],
    [
      5.0,
      3.0,
      1.6,
      0.2
    ],
    [
      4.9,
      3.1,
      1.5,
      0.1
    ],
    [
      5.8,
      2.7,
      5.1,
      1.9
    ],
    [
      6.4,
      3.2,
      4.5,
      1.5
    ],
    [
      5.4,
      3.4,
      1.7,
      0.2
    ],
    [
      5.8,
      2.7,
      4.1,
      1.0
    ],
    [
      7.7,
      3.8,
      6.7,
      2.2
    ],
    [
      4.8,
      3.0,
      1.4,
      0.3
    ],
    [
      5.5,
      2.6,
      4.4,
      1.2
    ],
    [
      6.5,
      3.0,
      5.2,
      2.0
    ],
    [
      5.0,
      3.6,
      1.4,
      0.2
    ],
    [
      6.7,
      3.1,
      4.7,
      1.5
    ],
    [
      6.4,
      3.1,
      5.5,
      1.8
    ],
    [
      7.4,
      2.8,
      6.1,
      1.9
    ],
    [
      6.4,
      3.2,
      5.3,
      2.3
    ],
    [
      4.9,
      3.6,
      1.4,
      0.1
    ],
    [
      6.3,
      2.9,
      5.6,
      1.8
    ],
    [
      6.3,
      3.3,
      4.7,
      1.6
    ],
    [
      5.7,
      2.8,
      4.1,
      1.3
    ],
    [
      5.3,
      3.7,
      1.5,
      0.2
    ],
    [
      5.1,
      3.8,
      1.9,
      0.4
    ],
    [
      5.6,
      3.0,
      4.1,
      1.3
    ],
    [
      6.0,
      2.9,
      4.5,
      1.5
    ],
    [
      5.0,
      2.3,
      3.3,
      1.0
    ],
    [
      6.1,
      3.0,
      4.6,
      1.4
    ],
    [
      6.0,
      2.2,
      4.0,
      1.0
    ],
    [
      6.5,
      2.8,
      4.6,
      1.5
    ],
    [
      6.9,
      3.1,
      4.9,
      1.5
    ],
    [
      5.9,
      3.0,
      4.2,
      1.5
    ],
    [
      6.3,
      2.5,
      4.9,
      1.5
    ],
    [
      4.5,
      2.3,
      1.3,
      0.3
    ],
    [
      5.2,
      2.7,
      3.9,
      1.4
    ],
    [
      6.4,
      2.8,
      5.6,
      2.2
    ],
    [
      4.6,
      3.4,
      1.4,
      0.3
    ],
    [
      6.7,
      3.3,
      5.7,
      2.5
    ],
    [
      6.6,
      3.0,
      4.4,
      1.4
    ],
    [
      5.2,
      4.1,
      1.5,
      0.1
    ],
    [
      6.9,
      3.1,
      5.4,
      2.1
    ],
    [
      5.6,
      2.5,
      3.9,
      1.1
    ],
    [
      5.5,
      2.4,
      3.8,
      1.1
    ]
  ]
}"""

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

array([0, 0, 2, 0, 2, 1, 2, 0, 2, 2, 0, 1, 0, 1, 1, 0, 0, 2, 0, 1, 0, 0,
       0, 2, 0, 1, 1, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 1, 0, 2, 2, 2, 0,
       0, 2, 0, 0, 1, 2, 0, 2, 2, 0, 0, 2, 2, 1, 2, 2, 1, 1, 2, 0, 0, 2,
       2, 1, 1, 2, 0, 0, 2, 0, 2, 1, 1, 2, 0, 0, 0, 0, 2, 1, 0, 1, 2, 0,
       1, 2, 0, 1, 2, 2, 2, 0, 2, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 2, 0, 2, 1, 0, 2, 1, 1])

In [47]:
model_info.model_uri

'runs:/30986c85c72a44cbaea472c1977a2aba/iris_model'

## Load the model back for pediction  as a generic python function model

### 2nd way- Inferencing and validating the model

In [49]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)
predictions = loaded_model.predict(X_test)
iris_features_name =  datasets.load_iris().feature_names
result = pd.DataFrame(X_test, columns=iris_features_name)
result["actual_class"] = y_test 
result["predicted_class"] = predictions

In [50]:
result

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,5.4,3.4,1.5,0.4,0,0
1,5.7,2.9,4.2,1.3,1,1
2,5.0,3.3,1.4,0.2,0,0
3,5.7,3.0,4.2,1.2,1,1
4,7.2,3.0,5.8,1.6,2,2
5,4.7,3.2,1.3,0.2,0,0
6,5.1,3.4,1.5,0.2,0,0
7,6.3,2.8,5.1,1.5,2,1
8,6.8,2.8,4.8,1.4,1,1
9,4.9,2.5,4.5,1.7,2,2


In [ ]:
r